# Get all family names

In [9]:
families = []

with open('Rfam.seed', encoding='ISO-8859-1') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith(f"#=GF AC"):
            families.append(line.strip().split("   ")[1])

# Given the Rfam seed extract all sequences from a family and write a multifasta for each family

In [ ]:
for family in families:
    sequences = []
    
    start_reading = False
    
+    with open('Rfam.seed', 'r',encoding='ISO-8859-1') as file:
        for line in file:
            if line.startswith(f"#=GF AC   {family}"):
                start_reading = True
                continue  
    
            if start_reading:
                if line.startswith("//"):
                    break
    
                if not line.startswith("#") and line.strip():
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        seq_id = parts[0]
                        sequence = parts[1]
                        sequences.append((seq_id, sequence)) 
    
    fo=open(f"family_fasta/{family}.fa", "w")
    
    for seq_id, sequence in sequences:
        fo.write(f">{seq_id}\n")
        fo.write(sequence.replace("-", "")+"\n")
    
    
    fo.close()

# run CD_HIT

In [ ]:
import os
for family in families:
    os.system(f"cd-hit-est -i family_fasta/{family}.fa -o out_cdhit/clustered_{family} > /dev/null")

# get the names of the representatives

In [ ]:
def extract_representative_ids(clstr_file):
    """
    Extracts representative sequence IDs from a CD-HIT .clstr file.
    
    :param clstr_file: Path to the CD-HIT .clstr output file.
    :return: List of representative sequence IDs.
    """
    representative_ids = []

    with open(clstr_file, 'r') as file:
        for line in file:
            if line.strip().endswith("*"):
                parts = line.strip().split(">")
                if len(parts) > 1:
                    seq_id = parts[1].split("...")[0] 
                    representative_ids.append(seq_id)

    return representative_ids

def rfam_to_rnafold_constraints(rfam_structure):
    """
    Convert Rfam consensus secondary structure notation to RNAfold constraint notation.

    Parameters:
    rfam_structure (str): The consensus secondary structure in Rfam format.

    Returns:
    str: The converted RNAfold constraint string.
    """
    rnafold_constraints = []

    # Mapping the Rfam symbols to RNAfold constraint symbols
    symbol_mapping = {
        '.': '.',   
        ',': '.',   
        '(': '(',    
        ')': ')',   
        '<': '|',   
        '>': '|',    
        '_': '.',    
        '-': '.',    
    }

    for char in rfam_structure:
        rnafold_constraints.append(symbol_mapping.get(char, '.'))

    return ''.join(rnafold_constraints)

def extract_conserved_positions(ss_cons_line, rf_line):
    conserved_positions = []

    for i, (rf, ss) in enumerate(zip(rf_line, ss_cons_line)):
        if rf.isupper(): 
            conserved_positions.append((i, rf, ss))
    return conserved_positions

In [ ]:
import re

for z,family in enumerate(families):
    if z%100==0:
        print(z, len(families))
    clstr_file = f"out_cdhit/clustered_{family}.clstr"
    representative_ids = extract_representative_ids(clstr_file)
    for id in representative_ids:
        ss_cons_line = ""
        rf_line = ""
        
        start_reading = False
        
        with open('Rfam.seed', encoding='ISO-8859-1') as file:
            for line in file:
                if line.startswith(f"#=GF AC   {family}"):
                    start_reading = True
                    continue  
                
                if start_reading:
                    if line.startswith(id) and line.strip():
                        parts = line.strip().split()
                        if len(parts) >= 2:
                            seq_id = parts[0]
                            sequence = parts[1]
                    
                    # Check for the ss_cons line and extract content after the tab
                    if line.startswith("#=GC SS_cons"):
                        ss_cons_line = line.split()[2].strip()
                    
                    # Check for the RF line and extract content after the tab
                    elif line.startswith("#=GC RF"):
                        rf_line = line.split()[2].strip()
                        
                        # Stop reading further as both lines are found
                        break
        file.close()
        
        rnafold_constraints = rfam_to_rnafold_constraints(ss_cons_line)
        
        conserved_positions = extract_conserved_positions(rnafold_constraints, rf_line)
        
        constraint_string = "." * len(sequence)
        for pos, _, element in conserved_positions:
                constraint_string = constraint_string[:pos] + element + constraint_string[pos + 1:]
        
        ungapped_seq=""
        ungapped_constraint_string=""
        
        for i, nucleotide in enumerate(sequence):
            if nucleotide!="-":
                ungapped_seq += nucleotide
                ungapped_constraint_string += constraint_string[i]
        
        header=seq_id.split("/")[0]
        
        with open(f"constraint_input/{header}.fa", "w") as f:
                f.write(f">{seq_id}\n{ungapped_seq}\n{ungapped_constraint_string}\n")

In [ ]:
#!cat *.fa > merged_sequences.fa

# run RNAfold 

In [2]:
import re
import os

def clean_rnafold_output(input_file, output_file):
    """
    Remove energy values from RNAfold output.
    
    :param input_file: Path to the RNAfold output file.
    :param output_file: Path to the cleaned output file.
    """
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            if re.search(r"\s\(-\d+\.\d+\)$", line):
                line = re.sub(r"\s\(-\d+\.\d+\)$", "", line)
            outfile.write(line)

In [3]:
files = os.listdir("constraint_input/")

In [4]:
for file in files:
    os.system(f"RNAfold --noPS -C < constraint_input/{file} > RNAfold_output/{file[:-3]}_folded.fa 2>/dev/null")

In [39]:
fo=open("RNAfold_output_no_energy/merged_sequences_folded_no_energy.fa", "w")

with open("RNAfold_output/merged_sequences_folded.fa") as file:
    lines=file.readlines()
    for i,line in enumerate(lines):
        if line.startswith(">"):
            fo.write(line)
            fo.write(lines[i+1])
            fo.write(lines[i+2].split()[0]+ "\n")

fo.close()

# Removing families with less than 5 members

In [41]:
from collections import defaultdict

In [35]:
dict_family_ids = defaultdict(list)

for family in families:    
    start_reading = False
    with open('Rfam.seed', 'r',encoding='ISO-8859-1') as file:
        for line in file:
            # Check for the starting line indicating the target family
            if line.startswith(f"#=GF AC   {family}"):
                start_reading = True
                continue 
    
            if start_reading:
                if line.startswith("//"):
                    break
    
                if not line.startswith("#") and line.strip():
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        seq_id = parts[0]
                        dict_family_ids[family].append(seq_id)  

In [42]:
remaining_values = [value for values in dict_family_ids.values() if len(values) >= 5 for value in values]

In [43]:
fo=open("output_RNAfold_5_members/rfam_new.fa", "w")

with open("RNAfold_output_no_energy/merged_sequences_folded_no_energy.fa") as f:
    lines=f.readlines()
    for i,line in enumerate(lines):
        if line.startswith(">"):
            id = line[1:].strip()
            if id in remaining_values:
                fo.write(line)
                fo.write(lines[i+1])
                fo.write(lines[i+2])
fo.close()